# Conditional Probability And Database Tables

Assume you have a following table T in your database.

| .. | Hair | Eyes | .. | 
|--|----------|----------|--|
| .. |   Blonde  |   Blue  | .. |
| .. |   Black  |   Blue  | .. |
| .. |   Blonde  |   Brown  | .. |
| .. |   Black  |   Brown  |  .. |

By counting the number of each distinct elements and dividing it with the cardinality, you obtain the joint probability of hair (=H) and eyes (=E). Let's assume the probability is the following:

```sql
SELECT
  Hair,
  Eyes,
  COUNT(*) / (SELECT COUNT(*) FROM T) AS P(H, E)
FROM T
GROUP BY Hair, Eyes;
```

| Hair | Eyes | $P(H, E)$ |
|----------|----------|----------|
|   Blonde  |   Blue  |   0.25  |
|   Black  |   Blue  |   0  |
|   Blonde  |   Brown  |   0.05  |
|   Black  |   Brown  |   0.7  |

Let's denote this new table T.
By taking the aggregation of the tables by one of the columns, we get probabilities for each of the attributes.

```sql
SELECT 
  Hair, 
  SUM(P(H, E)) AS P(H)
FROM T
GROUP BY Hair
```

| Hair | $P(H)$ |
|-|-|
|Blonde | 0.3 |
|Black | 0.7|

and 

```sql
SELECT 
  Eyes, 
  SUM(P(H, E)) AS P(E)
FROM T
GROUP BY Eyes
```

| Eyes | $P(E)$|
|-|-|
|Blue | 0.25 |
|Brown | 0.75 |

Let's denote these tables Thair and Teyes.
Because of the chain rule, $P(H | E) = P(H, E) / P(E)$, we can compute the conditional probability $P(H | E)$ (Given eye color E, the probability of having hair color H) by the following:


```sql
SELECT 
  Hair, 
  Eyes, 
  P(H, E), 
  P(E), 
  P(H, E) / P(E) AS P(H | E)
FROM T, Teyes
WHERE T.Eyes = Teyes.Eyes
```
| Hair | Eyes | $P(H, E)$ | $P(E)$ | $P(H \| E)$ |
|----------|----------|----------|--|----|
|   Blonde  |   Blue  |   0.25  | 0.25 | 1 |
|   Black  |   Blue  |   0  | 0.25 | 0 |
|   Blonde  |   Brown  |   0.05  | 0.75 | 1/15 |
|   Black  |   Brown  |   0.7  | 0.75 | 14/15 |

The cool thing about this is that this can be seen as a database operation not related to probability at all.
